In [1]:
#RNN

import tensorflow as tf
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.datasets import imdb
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import time

# Load IMDb dataset
max_features = 20000
maxlen = 80
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

# Define LSTM model
lstm_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 128),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile and train the model
lstm_model.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])


def train_model(model, X_train, y_train, X_test, y_test, model_name='RNN', dataset_name="IMDB", total_epoch=50, ea=False, eamonitor="val_loss"):
    tensorboard_callback = TensorBoard(log_dir='./../../Observation/'+model_name+'/'+model_name+dataset_name+'logs')
    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor=eamonitor, patience=10, min_delta=0.001, verbose=1)

    start_time = time.time()
    if ea:
        history = model.fit(X_train, y_train, epochs=total_epoch, batch_size=64, validation_data=(X_test, y_test), callbacks=[tensorboard_callback,early_stopping])
    else:
        history = model.fit(X_train, y_train, epochs=total_epoch, batch_size=64, validation_data=(X_test, y_test), callbacks=[tensorboard_callback])
    training_time = time.time() - start_time

    predictions = model.predict(X_test)

    return model, history, predictions, training_time

def test_model(model, X_test, y_test, model_name='RNN', dataset_name="IMDB", threshold=0.5):
    start_time = time.time()
    predictions = model.predict(X_test)
    inference_time = time.time() - start_time

    metrics = calculate_metrics(predictions, y_test, threshold)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] = inference_time
    metrics['Model type'] = model_name
    metrics['dataset'] = dataset_name

    print("Metrics:")
    for metric, value in metrics.items():
        print(f"{metric}: {value}")

    metrics_list = list(metrics.values())
    pd.DataFrame([metrics_list], columns=metrics.keys()).to_csv('./../../Observation/'+model_name+'/'+model_name+dataset_name+'metrics.csv', index=False)

    return metrics

def calculate_metrics(predictions, true_labels, threshold=0.5):
    binary_predictions = (predictions > threshold).astype('int32')  # Convert probabilities to binary predictions
    true_labels = true_labels.astype('int32')  # Ensure true labels are in integer format
    
    # Calculate accuracy
    accuracy = np.mean(binary_predictions == true_labels)
    
    # Calculate other metrics
    mse = mean_squared_error(true_labels, predictions)
    mae = mean_absolute_error(true_labels, predictions)
    rmse = np.sqrt(mse)
    f1 = f1_score(true_labels, binary_predictions)

    return {
        'Accuracy': accuracy,
        'MSE': mse,
        'MAE': mae,
        'RMSE': rmse,
        'F1 Score': f1
    }


# Train the model
model, history, predictions, training_time = train_model(lstm_model, x_train, y_train, x_test, y_test, model_name='RNN', dataset_name='IMDB', total_epoch=3, ea=True)

# Test the model
test_metrics = test_model(model, x_test, y_test, model_name='RNN', dataset_name='IMDB', threshold=0.5)


Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 50ms/step - accuracy: 0.7356 - loss: 0.5136 - val_accuracy: 0.8369 - val_loss: 0.3881
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 49ms/step - accuracy: 0.9087 - loss: 0.2410 - val_accuracy: 0.8360 - val_loss: 0.4094
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 49ms/step - accuracy: 0.9442 - loss: 0.1501 - val_accuracy: 0.8219 - val_loss: 0.4996
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step
782/782 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step
Metrics:
Accuracy: 0.5
MSE: 0.1393216402056211
MAE: 0.1964671763031643
RMSE: 0.3732581415128424
F1 Score: 0.8203389830508474
Training Time: 59.879998445510864
Inference Time: 6.305342435836792
Model type: RNN
dataset: IMDB
